In [1]:
from all_pipeline import load_model, predict_single_sentence, model_path, char_to_index, indicies_to_labels, max_len, batch_size, pickle, CharLSTM, remove_diactrics

In [2]:
def calculate_diacritic_error_rate(predictions, gold):
    """
    Calculate the diacritic error rate (DER) for a set of predictions and gold labels.

    :param predictions: A list of predictions.
    :param gold: A list of gold labels.
    :return: The diacritic error rate.
    """
    total = 0
    errors = 0
    for i in range(len(predictions)):
        total += 1
        if predictions[i] != gold[i]:
            errors += 1
    return errors / total

In [4]:
# load the predictions and gold labels from the csv files, with column label (second column in both files)
predictions = []
gold = []
with open('submission.csv', 'r') as f:
    for line in f:
        predictions.append(line.strip().split(',')[1])
with open('gold.csv', 'r') as f:
    for line in f:
        gold.append(line.strip().split(',')[1])
        
# calculate the DER
der = calculate_diacritic_error_rate(predictions, gold)
print(f'DER: {der * 100:.3f}%')
print(f'Accuracy: {(1 - der) * 100:.3f}%')

DER: 3.825%
Accuracy: 96.175%


In [4]:
# supress warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
model=load_model(model_path)

In [6]:
test_sentence = "ليس للوكيل بالقبض أن يبرأ المدين أو يهب الدين له أو يأخذ رهنا من المدين في مقابل الدين أو يقبل إحالته على شخص آخر لكن له أن يأخذ كفيلا لكن ليس له أن يأخذ كفيلا بشرط براءة الأصيل انظر المادة ( 648 ) ( الأنقروي ، الطحطاوي وصرة الفتاوى ، البحر ) ."
print(len(test_sentence))
print(test_sentence)
predicted_sentence = predict_single_sentence(model=model, original_sentence=test_sentence, max_len=max_len, char_to_index=char_to_index, indicies_to_labels=indicies_to_labels, batch_size=batch_size)
print(len(remove_diactrics([predicted_sentence])[0]))
print(predicted_sentence)

244
ليس للوكيل بالقبض أن يبرأ المدين أو يهب الدين له أو يأخذ رهنا من المدين في مقابل الدين أو يقبل إحالته على شخص آخر لكن له أن يأخذ كفيلا لكن ليس له أن يأخذ كفيلا بشرط براءة الأصيل انظر المادة ( 648 ) ( الأنقروي ، الطحطاوي وصرة الفتاوى ، البحر ) .
244
لَيْسَ لِلْوَكِيلِ بِالْقَبْضِ أَنْ يَبْرَأَ الْمَدِينُ أَوْ يَهَبَ الدَّيْنَ لَهُ أَوْ يَأْخُذَ رَهْنًا مِنْ الْمَدِينِ فِي مُقَابِلِ الدَّيْنِ أَوْ يُقْبَلَ إحَالَتَهُ عَلَى شَخْصٍ آخَرَ لَكِنْ لَهُ أَنْ يَأْخُذَ كَفِيلًا لَكِنْ لَيْسَ لَهُ أَنْ يَأْخُذَ كَفِيلًا بِشَرْطِ بَرَاءَةِ الْأَصِيلِ اُنْظُرْ الْمَادَّةَ ( 648 ) ( الْأُنْقِرْوِيَّ ، الطَّحْطَاوِيُّ وَصُرَّةُ الْفَتَاوَى ، الْبَحْرُ ) .
